In [1]:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# 
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [3]:
'''
Link to Google Drive root
'''

!mkdir -p drive
!google-drive-ocamlfuse drive
from google.colab import drive
drive.mount('/content/drive')
%cd '/content/drive/My Drive/NEE'

/bin/bash: google-drive-ocamlfuse: command not found
Mounted at /content/drive
/content/drive/My Drive/NEE


In [4]:
'''
Import required packages
'''

from sklearn.preprocessing import MinMaxScaler
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_squared_error

import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
import math
from scipy.stats import pearsonr

from tensorflow import keras
import keras
import keras.backend as K
from keras.models import Sequential
from keras.layers import TimeDistributed, Dense, LSTM, RepeatVector, Dropout, Input, Flatten,Concatenate
from keras.models import load_model
from keras.models import Model
from keras import optimizers
from tensorflow import optimizers

import matplotlib.pyplot as plt
import glob,os
import random
import os,shutil

In [44]:
# data preparation without lookback -- One to One prediction (o2o)
def data_prepare_o2o(dataset,scaler1, scaler2, shuffle):
    
    dataX, dataY = [], []
    scale_feature = scaler1.transform(dataset[:,:-1])
    scale_target = scaler2.transform(dataset[:,-1].reshape(-1, 1))
    for i in range(len(dataset)):
        feature = scale_feature[i]
        dataX.append(feature)
        target = scale_target[i]
        dataY.append(target)
        
    dataX, dataY = np.array(dataX), np.array(dataY)
    number_list = np.arange(0,len(dataX),1)
    random.shuffle(number_list) 
    dataX_new = []
    for index in number_list:
        dataX_new.append(dataX[index])
    dataY_new = []
    for index in number_list:
        dataY_new.append(dataY[index])

    if shuffle:
        return np.array(dataX_new), np.array(dataY_new)
    else:
        return np.array(dataX), np.array(dataY)

In [82]:
'''
Data load
'''

dataset = pd.read_csv(r'/content/drive/My Drive/NEE/data2_ML_new.csv',index_col = 0)

dynamic_feature_column = dataset.columns[8:18]
static_feature_column1 = dataset.columns[2:8]
static_feature_column2 = dataset.columns[19:]
target = dataset['NEE']

all_sites = dataset['Site'].unique()

result2 = pd.DataFrame(columns = all_sites, index = ['rmse_test', 'corr_test'])

In [83]:
# train and test split on all sites
from sklearn.model_selection import KFold
kf = KFold(n_splits=5, shuffle = True,random_state = 1) # n_splites can be changed based on different folds required; 
train_index_list = []
test_index_list = []
# train_index as gauged location for training, test_index as ungauged catchments for testing
for train_index, test_index in kf.split(all_sites):
    train_index_list.append(train_index)
    test_index_list.append(test_index)


In [84]:
### EXP2 ###
dynamic_dataset = dataset.drop(columns=['Unnamed: 0.1',  'VegType', 'lat', 'lon', 'timeStamp', 'Year',
           'Month', 'classKoppen', 'SWC', 'soilType', 'high_vegType', 'low_vegType',
           'high_vegCover', 'low_vegCover', 'Elavation'])

# LSTM cell: 32,64,96
# batch size:5, 10, 16, 32, 64, 128 -- try batch size 128 first -- over 15000 samples for each train

for i in range(len(train_index_list)):
    num_ite = i + 1 # count the number of folder
    print (num_ite)
    #site_dataset = dataset.loc[dataset['Site'] == train_index_list[i]]

    #dynamic_feature = site_dataset[dynamic_feature_column]
    #static_feature1 = site_dataset[static_feature_column1]
    #static_feature2 = site_dataset[static_feature_column2]

    # split into train and test sets
    #train_size = int(len(dynamic_dataset.values) * 0.75)
    #test_size = len(dynamic_dataset.values) - train_size
    #train, test = dynamic_dataset.values[0:train_size,:], dynamic_dataset.values[train_size:len(dynamic_dataset.values),:]

    train_sites = dynamic_dataset.Site.unique()[train_index_list[i]]
    train_dynamic_dataset = dynamic_dataset[dataset.Site.isin(train_sites)].drop(columns=['Site']).values

    test_sites = dynamic_dataset.Site.unique()[test_index_list[i]]
    test_dynamic_dataset = dynamic_dataset[dataset.Site.isin(test_sites)].drop(columns=['Site']).values

    scaler_x = StandardScaler()
    scaler_x.fit(train_dynamic_dataset[:,:-1])

    scaler_y = StandardScaler()
    scaler_y.fit(train_dynamic_dataset[:,-1].reshape(-1, 1))

    # train and test dataset
    trainX, trainY = data_prepare_o2o(train_dynamic_dataset, scaler_x, scaler_y, True)
    testX, testY = data_prepare_o2o(test_dynamic_dataset, scaler_x, scaler_y, True)
    
    '''
    Valina LSTM
    '''
    # define customer optimizer
    #RMSprop= optimizers.RMSprop(lr=0.0001)
    # define special callbacks
    reduce_lr = keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.3, patience=15, cooldown=30, min_lr=1e-8)
    # define early stop
    early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss', patience=20, verbose=0)

    model = Sequential()
    model.add(LSTM(32, input_shape=(trainX.shape[1], 1)))
    model.add(Dense(1))
    model.compile(loss='mean_squared_error', optimizer='adam')
    model.fit(trainX, trainY, epochs=300, validation_split=0.1, batch_size=128, callbacks=[reduce_lr, early_stopping], verbose=0)
    
    # save the model
    model.save(r'/content/drive/My Drive/NEE/nee_regional_%s.h5' % (num_ite)) 

    ### evaluate the overall performance ###
    trainPredict = model.predict(trainX)
    testPredict = model.predict(testX)
    
    trainPredict = scaler_y.inverse_transform(trainPredict)
    trainY = scaler_y.inverse_transform(trainY)
    testPredict = scaler_y.inverse_transform(testPredict)
    testY = scaler_y.inverse_transform(testY)

    # calculate root mean squared error
    trainScore = math.sqrt(mean_squared_error(trainY.squeeze(), trainPredict[:,0]))
    print('Train Score: %.2f RMSE' % (trainScore))
    testScore = math.sqrt(mean_squared_error(testY.squeeze(), testPredict[:,0]))
    print('Test Score: %.2f RMSE' % (testScore))
    # calculate correlation score
    corr_train, _ = pearsonr(trainY.squeeze(), trainPredict[:,0])
    print('Train corr: %.2f corr' % (corr_train))
    corr_test, _ = pearsonr(testY.squeeze(), testPredict[:,0])
    print('Test corr: %.2f corr' % (corr_test))

    ### for individual site ###
    for k in range(len(test_sites)):
        # make predictions
        test_single_dynamic_dataset = dynamic_dataset.loc[dynamic_dataset['Site'] == test_sites[k]].drop(columns=['Site']).values
        
        testX, testY = data_prepare_o2o(test_single_dynamic_dataset, scaler_x, scaler_y, True)
        testPredict = model.predict(testX)
        #trainPredict = scaler_y.inverse_transform(trainPredict)
        #trainY = scaler_y.inverse_transform(trainY)
        testPredict = scaler_y.inverse_transform(testPredict)
        testY = scaler_y.inverse_transform(testY)

        # calculate root mean squared error

        testScore = math.sqrt(mean_squared_error(testY.squeeze(), testPredict[:,0]))
        #print('Test Score: %.2f RMSE' % (testScore))
        # calculate correlation score
        corr_test, _ = pearsonr(testY.squeeze(), testPredict[:,0])
        result2[test_sites[k]] = [testScore,corr_test]
    
    #print (all_sites[i])

1
Train Score: 0.93 RMSE
Test Score: 1.83 RMSE
Train corr: 0.91 corr
Test corr: 0.59 corr
2
Train Score: 1.13 RMSE
Test Score: 2.12 RMSE
Train corr: 0.86 corr
Test corr: 0.52 corr
3
Train Score: 0.94 RMSE
Test Score: 1.93 RMSE
Train corr: 0.91 corr
Test corr: 0.60 corr
4
Train Score: 1.01 RMSE
Test Score: 2.19 RMSE
Train corr: 0.89 corr
Test corr: 0.53 corr
5
Train Score: 1.01 RMSE
Test Score: 1.96 RMSE
Train corr: 0.90 corr
Test corr: 0.51 corr


In [85]:
result2.to_csv(r'/content/drive/My Drive/NEE/result_exp2.csv')

In [ ]:
# SAVE model